<a target="_blank" href="https://colab.research.google.com/github/soheeyang/unified-prompt-selection/blob/main/notebooks/add_custom_prompt_selection_method.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>&nbsp;or in a local notebook.

In [ ]:
import os

try:
    # Setting up an environment for Google Colab.

    import google.colab, sys

    install_script = """#!/usr/bin/bash

    !(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
    cd /content && rm -rf /content/unified-prompt-selection
    git clone https://github.com/soheeyang/unified-prompt-selection.git unified-prompt-selection > install.log 2>&1
    pip install -r /content/unified-prompt-selection/requirements.txt >> install.log 2>&1
    pip install --upgrade google-cloud-storage >> install.log 2>&1"""

    with open("/content/install.sh", "w") as f:
        f.write(install_script)

    os.system("bash /content/install.sh")
    os.chdir("/content/unified-prompt-selection")
    sys.path.append("/content/unified-prompt-selection")
except ModuleNotFoundError as _:
    os.chdir("/content/unified-prompt-selection")
    os.system("pip install -r requirements.txt > install.log 2>&1")

# Adding Custom Prompt Selection Method

If you want to add a new prompt selection method, refer to the [../method/methods.py](../method/methods.py) module. Check other methods in this module, and create a new method according to the types and dimensions of input and output values.

||Task-wise Method|Instance-wise Method|
|:---------:|:-------:|:------------:|
|Input Variable|`template_prob`|`tensor_dict_prob`|
|Input Variable Type|torch.Tensor|torch.Tensor|
|Input Variable Size|[X, Y]|[T, X, Y]|
|Output Value Type|torch.Tensor|Tuple[List[float], List[int]]|
|Output Value Size|[]\(Scalar\)|([X], [X])|

- **T** : The number of prompts
- **X** : The number of instances
- **Y** : The number of answer choices

The Instance-wise Method has two lists as output values. The first list should contain the results of calculated instance-wise prompt selection scores using the added method, and the second list should contain the indices of the selected prompts for each instance.

The newly added method is utilized in the `get_*_wise_ps_scores` function in the [../method/score.py](../method/score.py) module.

```python
methodFuncMap = {
    'MI': get_mi_g if one_hot else get_mi,
    'GE': get_ge if one_hot else get_ge_m,
    'LE': get_le,
    'MDL': get_le,
    'PPL': get_ppl,
    'ZLP': get_zlp,
    'ZPM': get_zpm,
    'ZMV': get_zmv,
}
```

When adding a method, follow the format of the method in the methodFuncMap inside the `get_*_wise_ps_result` function. Additionally, add a yaml file representing the new method to [`../conf/method`](../conf/method/) directory. Refer to the yaml file format of other methods in `../conf/method`

The following example demonstrates the process of adding a new method by adjusting the weights of GE based on the default configuration, creating template_prob and tensor_dict_prob variables, and examining their shapes and types. The newly added method modifies GE weights, and this process is illustrated step by step.

The default configuration focuses on the 'glue-sst2-validation' dataset, consisting of 872 instances(X) and 2 answer choices(Y). There are 100 prompts(T) based on base_prompts.

In [ ]:
# Preparing Base Configuration
import os
import pickle

get_config = """
import hydra
import pickle

@hydra.main(version_base=None, config_path="conf", config_name="config")
def main(cfg):
    with open("cfg.pkl", "wb") as f:
        pickle.dump(cfg, f)

if __name__ == "__main__":
    main()
"""

with open("get_config.py", "w") as f:
    f.write(get_config)
os.system("python get_config.py")
os.remove("get_config.py")

with open("cfg.pkl", "rb") as f:
    cfg = pickle.load(f)
os.remove("cfg.pkl")

# Check `template_prob` and `tensor_dict_prob`
from method.postprocessor import PostProcessor

post_processor = PostProcessor(cfg)
tensor_dict, targets = post_processor.get_tensor_dict_and_targets()
template_prob, tensor_dict_prob = tensor_dict['prob'][0], tensor_dict['prob']

input_var_info = f'''
{'-'*40}
template_prob Dimension: {template_prob.dim()}
template_prob Size: [{template_prob.size(0)}, {template_prob.size(1)}]

tensor_dict_prob Dimension: {tensor_dict_prob.dim()}
tensor_dict_prob Size: [{tensor_dict_prob.size(0)}, {tensor_dict_prob.size(1)}, {tensor_dict_prob.size(2)}]
{'-'*40}
'''

print(input_var_info)

Here are some prompt selection method functions from [../method/methods.py](../method/methods.py). Let's create new methods, MI with triple GE weight (3GE_MI) and MI_L with triple GE weight (3GE_MI_L), considering the output format of existing functions.

In [ ]:
import torch
from torch import Tensor
from typing import Tuple, List


def safe_log(prob: Tensor) -> Tensor:
    return torch.log(prob + 1e-7)

def get_entropy(prob: Tensor, sum_axis: int, keepdims: bool = False) -> Tensor:
    return -(prob * safe_log(prob)).sum(axis=sum_axis, keepdims=keepdims)

def get_le(template_prob: Tensor) -> Tensor:
    return get_entropy(template_prob, sum_axis=-1).mean()

def get_ge_m(template_prob: Tensor, X_axis: int = 0, Y_axis: int = -1, keepdims: bool = False) -> Tensor:
    d_prob = template_prob.mean(axis=X_axis, keepdims=keepdims)
    return get_entropy(d_prob, sum_axis=Y_axis, keepdims=keepdims)

def get_mi(template_prob: Tensor) -> Tensor:
    ge_m = get_ge_m(template_prob)
    mdl_m = get_le(template_prob)
    return (ge_m - mdl_m)

def get_mi_l(tensor_dict_prob: Tensor) -> Tuple[List[float], List[int]]:
    tensor_dict_prob = tensor_dict_prob.transpose(0, 1)  # [X, T, Y]
    ge_m = get_ge_m(tensor_dict_prob, X_axis=0, Y_axis=-1)

    X = tensor_dict_prob.size(0)
    mi_als, selected_prompt_indices = [], []
    for i in range(X):
        instance_prob = tensor_dict_prob[i]  # [T, Y]
        mdl = get_entropy(instance_prob, -1)
        mi_al = ge_m - mdl
        mi_als.append(mi_al.max().item())
        selected_prompt_indices.append(mi_al.argmax().item())
    return mi_als, selected_prompt_indices

The new methods are created by simply utilizing the existing functions as follows.

In [ ]:
def get_3ge_mi(template_prob: Tensor) -> Tensor:
    ge_m = 3 * get_ge_m(template_prob)
    mdl_m = get_le(template_prob)
    return (ge_m - mdl_m)

def get_3ge_mi_l(tensor_dict_prob: Tensor) -> Tuple[List[float], List[int]]:
    tensor_dict_prob = tensor_dict_prob.transpose(0, 1)  # [X, T, Y]
    ge_m = 3 * get_ge_m(tensor_dict_prob, X_axis=0, Y_axis=-1)

    X = tensor_dict_prob.size(0)
    mi_als, selected_prompt_indices = [], []
    for i in range(X):
        instance_prob = tensor_dict_prob[i]  # [T, Y]
        mdl = get_entropy(instance_prob, -1)
        mi_al = ge_m - mdl
        mi_als.append(mi_al.max().item())
        selected_prompt_indices.append(mi_al.argmax().item())
    return mi_als, selected_prompt_indices

Let's verify the output of the newly added methods.

In [ ]:
output_of_task_wise_method = get_3ge_mi(template_prob)
output_of_instance_wise_method = get_3ge_mi_l(tensor_dict_prob)

output_value_info = f'''
{'-'*200}
Task-wise Method Type: {type(output_of_task_wise_method)}
Task-wise Method Output: {output_of_task_wise_method}


Instance-wise Method Type: ( {type(output_of_instance_wise_method)} [ {type(output_of_instance_wise_method[0])}, {type(output_of_instance_wise_method[1])} ] )
Instance-wise Method Size: [ {len(output_of_instance_wise_method[0])}, {len(output_of_instance_wise_method[1])} ]

Instance-wise Method Output, Index == 0: \n{output_of_instance_wise_method[0]}

Instance-wise Method Output, Index == 1: \n{output_of_instance_wise_method[1]}
{'-'*200}
'''

print(output_value_info)

As mentioned earlier, these added methods must be included in the methodFuncMap of the get_task_wise_ps_result and get_instance_wise_ps_result functions in [../method/score.py](../method/score.py).

```python
# methodFuncMap in `get_task_wise_ps_result`
methodFuncMap = {
    'MI': get_mi_g if one_hot else get_mi,
    'GE': get_ge if one_hot else get_ge_m,
    'LE': get_le,
    'MDL': get_le,
    'PPL': get_ppl,
    'ZLP': get_zlp,
    'ZPM': get_zpm,
    'ZMV': get_zmv,
    '3GE_MI': get_3ge_mi
}

# methodFuncMap in `get_instance_wise_ps_result`
methodFuncMap = {
    'PPL': get_i_ppl,
    'MDL': get_mdl,
    'MI': get_mi_gl if one_hot else get_mi_l,
    '3GE_MI': get_3ge_mi_l
}
```

Lastly, include the YAML files for the new methods in [../conf/method](../conf/method/), and you can use them directly through the run_prompt_selection.py script.

```yaml
## '../conf/method/3GE_MI.yaml'

# Pass the prompt selection method to use as an argument.
method: '3GE_MI'

# If passed, the GE is computed with one hot P(y|x,t).
one_hot: false

# If passed, a prompt is selected for each x.
select_for_each_x: false
```

```yaml
## '../conf/method/3GE_MI_L.yaml'

# Pass the prompt selection method to use as an argument.
method: '3GE_MI'

# If passed, the GE is computed with one hot P(y|x,t).
one_hot: false

# If passed, a prompt is selected for each x.
select_for_each_x: true
```

```bash
python run_prompt_selection \
    method=3GE_MI \
    select_for_each_x=false,true
```